# Substitution Ciphers

In [1]:
alphabet = 'abcdefghijklmnopqrstuvwxyz'
key =      'jsuyfhkpicomxrqatlbvznewgd'


def substitute(text, substitute_what, substitute_by):
    result = ''
    for symbol in text.lower():
        if symbol in substitute_what:
            result += substitute_by[
                substitute_what.index(symbol)
            ]
        else:
            result += symbol

    return result


def encode(plaintext):
    return substitute(plaintext, alphabet, key)


def decode(ciphertext):
    return substitute(ciphertext, key, alphabet)


message = 'the quick brown fox jumps over the lazy dog'
code = encode(message)
print(code)
print(decode(code))

vpf tziuo slqer hqw czxab qnfl vpf mjdg yqk
the quick brown fox jumps over the lazy dog


Cyclic shift of the alphabet:

In [2]:
alphabet = 'abcdefghijklmnopqrstuvwxyz'
key = alphabet[3:] + alphabet[:3]

print(alphabet)
print(key)

abcdefghijklmnopqrstuvwxyz
defghijklmnopqrstuvwxyzabc


Trying all cyclic shifts:

In [3]:
ciphertext = 'kyv wzmv sfozex nzqriuj aldg hlztbcp'
for shift in range(26):
    key = alphabet[shift:] + alphabet[:shift]
    print(decode(ciphertext))

kyv wzmv sfozex nzqriuj aldg hlztbcp
jxu vylu renydw mypqhti zkcf gkysabo
iwt uxkt qdmxcv lxopgsh yjbe fjxrzan
hvs twjs pclwbu kwnofrg xiad eiwqyzm
gur svir obkvat jvmneqf whzc dhvpxyl
ftq ruhq najuzs iulmdpe vgyb cguowxk
esp qtgp mzityr htklcod ufxa bftnvwj
dro psfo lyhsxq gsjkbnc tewz aesmuvi
cqn oren kxgrwp frijamb sdvy zdrltuh
bpm nqdm jwfqvo eqhizla rcux ycqkstg
aol mpcl ivepun dpghykz qbtw xbpjrsf
znk lobk hudotm cofgxjy pasv waoiqre
ymj knaj gtcnsl bnefwix ozru vznhpqd
xli jmzi fsbmrk amdevhw nyqt uymgopc
wkh ilyh eralqj zlcdugv mxps txlfnob
vjg hkxg dqzkpi ykbctfu lwor swkemna
uif gjwf cpyjoh xjabset kvnq rvjdlmz
the five boxing wizards jump quickly
sgd ehud anwhmf vhyzqcr itlo pthbjkx
rfc dgtc zmvgle ugxypbq hskn osgaijw
qeb cfsb ylufkd tfwxoap grjm nrfzhiv
pda bera xktejc sevwnzo fqil mqeyghu
ocz adqz wjsdib rduvmyn ephk lpdxfgt
nby zcpy vircha qctulxm dogj kocwefs
max ybox uhqbgz pbstkwl cnfi jnbvder
lzw xanw tgpafy oarsjvk bmeh imaucdq


# One-time Pad

In [ ]:
from itertools import product


def encode(plain_text, private_key):
    assert plain_text in {0, 1} and private_key in {0, 1}
    return plain_text ^ private_key


def decode(cypher_text, private_key):
    assert cypher_text in {0, 1} and private_key in {0, 1}
    return cypher_text ^ private_key


for plaintext, private_key in product({0, 1}, repeat=2):
    ciphertext = encode(plaintext, private_key)
    print(f'key: {private_key}, '
          f'plaintext: {plaintext}, '
          f'ciphertext: {ciphertext}, '
          f'decoded: {decode(ciphertext, private_key)}')

## Many Time Pad Attack

In [4]:
def to_hex(plain_text):
    hex_codes = []
    for symbol in plain_text:
        hex_code = hex(ord(symbol)).replace('0x', '')
        if len(hex_code) == 1:
            hex_code = '0' + hex_code
        hex_codes.append(hex_code)
    return ''.join(hex_codes)


def to_str(hex_code):
    if hex_code:
        return chr(int(hex_code[:2], base=16)) + to_str(hex_code[2:])
    return ''


message = 'Hello World'
print(f'hex of {message} is: {to_hex(message)}')

code = '736f6d65206d657373616765'
print(f'str of {code} is: {to_str(code)}')


def bitwise_xor(first_text, second_text):
    assert len(first_text) == len(second_text)
    return ''.join(format(int(s1, 16) ^ int(s2, 16), '01x')
                   for s1, s2 in zip(first_text, second_text))


message = 'secret message'
key =     'my secret keys'
print(f'hex of {message} is: {to_hex(message)}')
print(to_hex(key))

ciphertext = bitwise_xor(to_hex(message), to_hex(key))
print('ciphertext:', ciphertext)

recovered_message = to_str(bitwise_xor(ciphertext, to_hex(key)))
print('recovered message:', recovered_message)


message1 = 'steal the secret'
message2 = 'the boy the girl'
key = 'supersecretverys'

ciphertext1 = bitwise_xor(to_hex(message1), to_hex(key))
ciphertext2 = bitwise_xor(to_hex(message2), to_hex(key))

xor_ciphertexts = bitwise_xor(ciphertext1, ciphertext2)
xor_messages = bitwise_xor(to_hex(message1), to_hex(message2))

print(xor_ciphertexts)
print(xor_messages)

if xor_ciphertexts == xor_messages:
    print('Xor of the ciphertexts is the same as xor of messages')
else:
    print('Xor of the ciphertexts differs from the xor of messages')


def try_guessing_substring(substring, message_length, xor_messages):
    good_guesses = []
    for pos in range(message_length - len(substring) + 1):
        guess = to_hex(chr(0) * pos + substring +
                       chr(0) * (message_length - len(substring) - pos))
        other_message_part = to_str(
            bitwise_xor(guess, xor_messages)
        )[pos:pos + len(substring)]
        good_guess = True
        for i in range(len(other_message_part)):
            if not other_message_part[i].isalpha() and \
                    not other_message_part[i].isspace():
                good_guess = False
                break
        if good_guess:
            good_guesses.append((guess, pos, other_message_part))

    print('Good guesses:')
    for guess in good_guesses:
        print(f'position {guess[1]}:\n'
              f'       one message part: \"{substring}\"\n'
              f' the other message part: \"{guess[2]}\"')


try_guessing_substring(' the ', len(message1), xor_messages)

try_guessing_substring('oy the ', len(message1), xor_messages)

hex of Hello World is: 48656c6c6f20576f726c64
str of 736f6d65206d657373616765 is: some message
hex of secret message is: 736563726574206d657373616765
6d7920736563726574206b657973
ciphertext: 1e1c430100175208115318041e16
recovered message: secret message
071c00410e4f0d4811481645041b1718
071c00410e4f0d4811481645041b1718
Xor of the ciphertexts is the same as xor of messages
Good guesses:
position 5:
       one message part: " the "
 the other message part: "oy th"
position 7:
       one message part: " the "
 the other message part: "he se"
Good guesses:
position 5:
       one message part: "oy the "
 the other message part: " the se"


# RSA

In [5]:
from math import gcd
p, q = 5915587277, 2860486313

n = p * q
print(f'n={n}')

phi = (p - 1) * (q - 1)
print(f'phi={phi}')
e = 3
print(f'e={e}')
assert gcd(phi, e) == 1

d = pow(e, -1, phi)
print(f'd={d}')
assert d * e % phi == 1


def encode(m):
    assert 0 <= m < n
    return pow(m, e, n)


message = 92616855427
print(f'message: {message}')
cipher_text = encode(message)


def decode(c):
    return pow(c, d, n)


print(f'decoded: {decode(cipher_text)}')

n=16921456439215439701
phi=16921456430439366112
e=3
d=11280970953626244075
message: 92616855427
decoded: 92616855427
